In [1]:
# Install d2l
!pip install d2l==0.17.2

     |████████████████████████████████| 82 kB 403 kB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 9.9 MB 46.7 MB/s 
     |████████████████████████████████| 11.6 MB 47.8 MB/s 
     |████████████████████████████████| 61 kB 7.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the p

In [2]:
# Import Library
%matplotlib inline
import random
import tensorflow as tf
from d2l import tensorflow as d2l

In [3]:
# Generating the Dataset
def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise."""
    X = tf.zeros((num_examples, w.shape[0]))
    X += tf.random.normal(shape=X.shape)
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b
    y += tf.random.normal(shape=y.shape, stddev=0.01)
    y = tf.reshape(y, (-1, 1))
    return X, y

true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [4]:
print('features:', features[0],'\nlabel:', labels[0])

features: tf.Tensor([0.45174658 0.5355845 ], shape=(2,), dtype=float32) 
label: tf.Tensor([3.2837203], shape=(1,), dtype=float32)


In [5]:
d2l.set_figsize()
# The semicolon is for displaying the plot only
d2l.plt.scatter(features[:, (1)].numpy(), labels.numpy(), 1);

ImportError: ignored

<Figure size 252x180 with 1 Axes>

In [6]:
#Reading the Dataset
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = tf.constant(indices[i: min(i + batch_size, num_examples)])
        yield tf.gather(features, j), tf.gather(labels, j)

In [7]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tf.Tensor(
[[ 0.15766202  0.5871559 ]
 [ 0.46258482 -0.18574502]
 [ 1.0803657  -0.52380997]
 [-0.321619    0.439967  ]
 [-1.3511119  -0.087855  ]
 [ 0.1393825   0.28925297]
 [-0.151499    0.7722258 ]
 [-0.17250608 -0.06026872]
 [-0.38722095 -0.9190756 ]
 [ 1.5236034   1.0577176 ]], shape=(10, 2), dtype=float32) 
 tf.Tensor(
[[2.5237515]
 [5.7427607]
 [8.130285 ]
 [2.068609 ]
 [1.7933838]
 [3.4956038]
 [1.2583888]
 [4.054906 ]
 [6.5438275]
 [3.65544  ]], shape=(10, 1), dtype=float32)


In [8]:
# Initializing Model Parameters
w = tf.Variable(tf.random.normal(shape=(2, 1), mean=0, stddev=0.01),
                trainable=True)
b = tf.Variable(tf.zeros(1), trainable=True)

In [9]:
# Defining the Model
def linreg(X, w, b): 
    """The linear regression model."""
    return tf.matmul(X, w) + b

In [10]:
# Defining the Loss Function
def squared_loss(y_hat, y):
    """Squared loss."""
    return (y_hat - tf.reshape(y, y_hat.shape)) ** 2 / 2

In [11]:
# Defining the Optimization Algorithm
def sgd(params, grads, lr, batch_size): 
    """Minibatch stochastic gradient descent."""
    for param, grad in zip(params, grads):
        param.assign_sub(lr*grad/batch_size)

In [12]:
# Training
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with tf.GradientTape() as g:
            l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on l with respect to [`w`, `b`]
        dw, db = g.gradient(l, [w, b])
        # Update parameters using their gradient
        sgd([w, b], [dw, db], lr, batch_size)
    train_l = loss(net(features, w, b), labels)
    print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_l)):f}')

epoch 1, loss 0.040886
epoch 2, loss 0.000157
epoch 3, loss 0.000051


In [13]:
print(f'error in estimating w: {true_w - tf.reshape(w, true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: [ 0.00019681 -0.00085545]
error in estimating b: [0.00055122]
